<center>
  <div style="background-color:#334155; color:#e2e8f0; padding:2rem; border-radius:1rem; text-align:center; font-family:sans-serif; margin-bottom:2rem;">
    <h1 style="color:#58a6ff; margin-bottom:0.5rem;">Ozan MÖHÜRCÜ</h1>
    <h2 style="color:#cbd5e1; font-weight:400; font-size:1.25rem; margin-top:0; margin-bottom:1.5rem;">Data Analyst | Data Scientist</h2>
    <a href="https://www.linkedin.com/in/ozanmhrc/" target="_blank" rel="noopener noreferrer">
      <img src="https://img.shields.io/badge/LinkedIn-0A66C2?style=for-the-badge&logo=linkedin&logoColor=white" alt="LinkedIn Profile">
    </a>
    <a href="https://github.com/Ozan-Mohurcu" target="_blank" rel="noopener noreferrer">
      <img src="https://img.shields.io/badge/GitHub-171515?style=for-the-badge&logo=github&logoColor=white" alt="GitHub Profile">
    </a>
    <a href="https://ozan-mohurcu.github.io/" target="_blank" rel="noopener noreferrer">
      <img src="https://img.shields.io/badge/Portfolio-6A1B9A?style=for-the-badge&logo=google-chrome&logoColor=white" alt="Portfolio Website">
    </a>
  </div>
</center>

<div style="background-color: #111827; color: #F3F4F6; border-left: 10px solid #3B82F6; padding: 25px; margin: 20px 0px; border-radius: 8px; box-shadow: 0 4px 8px 0 rgba(0,0,0,0.2); text-align: justify;">
  <h1 style="color: #60A5FA; text-align: center; margin-top: -10px; margin-bottom: 25px;">NFL Big Data Bowl 2026: An End-to-End Play Prediction Strategy</h1>
  
  <h2 style="color: #60A5FA; border-bottom: 2px solid #3B82F6; padding-bottom: 10px;">Project Overview & Objectives</h2>
  <p>Welcome to this comprehensive walkthrough for the NFL Big Data Bowl 2026! The core challenge of this competition is to predict the future locations (x, y coordinates) of players on the field based on a sequence of tracking data. This is a fascinating problem that blends time-series analysis, physics-based modeling, and understanding complex player interactions.</p>
  <p><b>Our strategic approach in this notebook will be:</b></p>
  <ul>
    <li><b>Foundation in Physics:</b> We'll start by building a simple, yet powerful, constant velocity baseline model. This gives us a solid benchmark and serves as the foundation for our advanced model.</li>
    <li><b>Advanced Feature Engineering:</b> We will extract a rich set of features, capturing player kinematics, geometry relative to the ball, and historical movement patterns (lags, rolling stats).</li>
    <li><b>Modeling Player Interactions (GNN-lite):</b> A player's movement is heavily influenced by those around them. We'll implement a lightweight Graph Neural Network (GNN) concept to create "neighbor embeddings," summarizing the state of nearby allies and opponents.</li>
    <li><b>Residual Prediction with CatBoost:</b> Instead of predicting the absolute final coordinates, we will train our model to predict the <i>error (or residual)</i> of our physics baseline. This is a powerful technique that helps the model focus on learning the complex, non-linear dynamics that the simple model misses.</li>
    <li><b>Robust Validation:</b> We will use GroupKFold cross-validation to ensure our model generalizes well and that we don't have data leakage between training and validation sets.</li>
  </ul>
</div>

1. Setup and Configuration
<div style="background-color: #f0f8ff; color: #333; border-left: 5px solid #4CAF50; padding: 15px; margin: 20px 0px; border-radius: 5px;">
<p>First, let's set up our environment. We'll import the necessary libraries and define our global configuration parameters. This includes settings for our model, feature engineering, and cross-validation strategy. Encapsulating these in a single location makes the notebook cleaner and easier to modify.</p>
</div>

In [ ]:
import os
import gc
import math
import pickle
import warnings
from pathlib import Path

import numpy as np
import pandas as pd
from multiprocessing import Pool as MP, cpu_count

from sklearn.model_selection import KFold, GroupKFold
from sklearn.metrics import mean_squared_error
from catboost import CatBoostRegressor, Pool as CatPool
from tqdm.auto import tqdm

# Ignore harmless warnings
warnings.filterwarnings("ignore")

# --- Configuration Block ---

class CFG:
    # Paths
    BASE_DIR = Path("/kaggle/input/nfl-big-data-bowl-2026-prediction")
    SAVE_DIR = Path("/kaggle/working")
    
    # Data Specs
    N_WEEKS = 18
    
    # Model Parameters (CatBoost)
    # Using a high number of iterations with early stopping is a robust approach.
    ITERATIONS = 20000
    LEARNING_RATE = 0.08
    DEPTH = 8
    L2_REG = 3.0
    EARLY_STOPPING_ROUNDS = 500
    
    # CV Strategy
    N_FOLDS = 5
    USE_GROUP_KFOLD = True  # Highly recommended to prevent data leakage
    SEED = 42
    
    # GNN-lite (Neighbor Embedding) Parameters
    K_NEIGHBORS = 6       # How many nearest neighbors to consider for each player
    RADIUS_LIMIT = 30.0   # Max distance (yards) to look for neighbors
    TAU = 8.0             # Temperature parameter for softmax weighting (controls influence falloff)
    
    # Environment
    # Automatically detect if a GPU is available for CatBoost
    USE_GPU = bool(os.environ.get("CUDA_VISIBLE_DEVICES", ""))

# Create save directory if it doesn't exist
CFG.SAVE_DIR.mkdir(exist_ok=True)

print(f"Running with GPU support: {CFG.USE_GPU}")
print(f"CPU cores available: {cpu_count()}")

2. Data Loading & Initial Exploration
<div style="background-color: #f0f8ff; color: #333; border-left: 5px solid #4CAF50; padding: 15px; margin: 20px 0px; border-radius: 5px;">
<p>The training data is split into 18 weekly files. We'll write a utility function to load all of them in parallel, which significantly speeds up the process. After loading, we'll perform a quick sanity check to understand the structure and volume of our dataset.</p>
<h4>Understanding the Coordinate System</h4>
<p>It's critical to correctly interpret the geometry of the data. In the NFL's system:</p>
<ul>
<li>The field is 120 yards long (including end zones) and 53.3 yards wide.</li>
<li><code>(0, 0)</code> is the corner of the home team's end zone.</li>
<li><code>x</code> represents the yard line, from 0 to 120.</li>
<li><code>y</code> represents the distance from the sideline, from 0 to 53.3.</li>
<li>Direction (<code>dir</code>) and Orientation (<code>o</code>) are in degrees, where <b>0 degrees points straight down the field towards the opponent's end zone (+Y direction in physics calculations, but this needs careful conversion)</b>. This is a common source of bugs! We will standardize this.</li>
</ul>
</div>

In [ ]:
def load_week_data(week_num: int):
    """Loads the input and output data for a single week."""
    input_path = CFG.BASE_DIR / f"train/input_2023_w{week_num:02d}.csv"
    output_path = CFG.BASE_DIR / f"train/output_2023_w{week_num:02d}.csv"
    return pd.read_csv(input_path), pd.read_csv(output_path)

def load_all_training_data():
    """Loads all 18 weeks of training data in parallel."""
    print("Loading all training data using parallel processing...")
    # Use a multiprocessing pool to load files concurrently
    with MP(min(cpu_count(), CFG.N_WEEKS)) as pool:
        results = list(tqdm(pool.imap(load_week_data, range(1, CFG.N_WEEKS + 1)), total=CFG.N_WEEKS))
    
    # Concatenate the results into two large DataFrames
    train_input_df = pd.concat([res[0] for res in results], ignore_index=True)
    train_output_df = pd.concat([res[1] for res in results], ignore_index=True)
    
    print(f"Total train input rows:  {len(train_input_df):,}")
    print(f"Total train output rows: {len(train_output_df):,}")
    
    # Free up memory
    del results
    gc.collect()
    
    return train_input_df, train_output_df

# Execute the loading process
train_input_df, train_output_df = load_all_training_data()

print("\nSample of Input Data:")
display(train_input_df.head())
print("\nSample of Output Data:")
display(train_output_df.head())

3. Feature Engineering Pipeline
<div style="background-color: #1F2937; color: #D1D5DB; border-left: 10px solid #F59E0B; padding: 25px; margin: 20px 0px; border-radius: 8px;">
<h3 style="color: #FBBF24; border-bottom: 2px solid #F59E0B; padding-bottom: 10px;">The Art and Science of Feature Creation</h3>
<p>This is where we add the most value. Raw tracking data is just a starting point. We need to create features that explicitly provide the model with information about a player's physical state, their intent, their movement history, and their relationship with other players on the field. We'll build this in a modular pipeline.</p>
</div>

3.1. Core Physics and Geometry Features
<div style="background-color: #f0f8ff; color: #333; border-left: 5px solid #4CAF50; padding: 15px; margin: 20px 0px; border-radius: 5px;">
<p>Here, we convert raw stats into a more meaningful physical representation. A key step is standardizing the angle convention. The NFL's dir is unconventional for physics. We will convert it to a standard mathematical angle where 0 degrees is the +X axis, which allows us to use sin and cos correctly to decompose vectors.</p>
<ul>
<li><b>Player Attributes:</b> Convert height to inches and calculate Body Mass Index (BMI).</li>
<li><b>Standardized Kinematics:</b> Decompose speed (<code>s</code>) and acceleration (<code>a</code>) into their X and Y components (<code>velocity_x</code>, <code>velocity_y</code>, etc.) using the corrected heading.</li>
<li><b>Target Geometry:</b> Calculate the player's distance and angle to the predicted ball landing spot. This is a powerful indicator of a player's objective.</li>
<li><b>Advanced Physics:</b> Features like momentum and kinetic energy can help the model understand the "cost" of changing direction.</li>
</ul>
</div>

In [ ]:
def convert_height_to_inches(h_str):
    """Converts height string 'feet-inches' to total inches."""
    try:
        feet, inches = map(int, str(h_str).split('-'))
        return float(feet) * 12.0 + float(inches)
    except:
        return np.nan

def add_physics_features(df: pd.DataFrame) -> pd.DataFrame:
    """Engineers physics-based and geometric features."""
    df = df.copy()
    
    # --- Player Attributes ---
    df['height_inches'] = df['player_height'].apply(convert_height_to_inches)
    df['bmi'] = (df['player_weight'] / (df['height_inches']**2)) * 703.0

    # --- Standardize Angles and Decompose Vectors ---
    # The 'dir' column is clockwise from the +y axis. Let's convert to standard
    # counter-clockwise from +x axis for easier trigonometry.
    # Standard Angle (rad) = pi/2 - dir(rad)
    dir_rad = np.radians(df['dir'].fillna(0.0))
    std_angle_rad = np.pi/2 - dir_rad
    
    df['heading_x'] = np.cos(std_angle_rad)
    df['heading_y'] = np.sin(std_angle_rad)

    s = df['s'].fillna(0.0)
    a = df['a'].fillna(0.0)
    df['velocity_x'] = s * df['heading_x']
    df['velocity_y'] = s * df['heading_y']
    df['acceleration_x'] = a * df['heading_x']
    df['acceleration_y'] = a * df['heading_y']

    # --- Geometry relative to ball landing spot ---
    dx_ball = df['ball_land_x'] - df['x']
    dy_ball = df['ball_land_y'] - df['y']
    dist_ball = np.sqrt(dx_ball**2 + dy_ball**2)
    
    df['dist_to_ball'] = dist_ball
    # Angle to ball in radians, using standard atan2
    df['angle_to_ball_rad'] = np.arctan2(dy_ball, dx_ball)
    
    # Unit vector towards the ball
    ball_unit_x = dx_ball / (dist_ball + 1e-6)
    ball_unit_y = dy_ball / (dist_ball + 1e-6)
    
    # Velocity component towards the ball (dot product of velocity and ball unit vector)
    df['velocity_towards_ball'] = df['velocity_x'] * ball_unit_x + df['velocity_y'] * ball_unit_y
    # Alignment with ball path (dot product of heading and ball unit vector)
    df['heading_alignment_ball'] = df['heading_x'] * ball_unit_x + df['heading_y'] * ball_unit_y

    # --- Other Physics and Role-based Features ---
    w = df['player_weight'].fillna(200.0) # Impute with average weight
    df['kinetic_energy'] = 0.5 * w * (s**2)
    df['momentum'] = w * s
    
    df['is_offense'] = (df['player_side'] == 'Offense').astype(int)
    df['is_targeted_receiver'] = (df['player_role'] == 'Targeted Receiver').astype(int)

    return df

print("Applying physics and geometry feature engineering...")
train_input_df = add_physics_features(train_input_df)
print("Done. New features added.")

3.2. Sequential Features (Lags & Rolling Stats)
<div style="background-color: #f0f8ff; color: #333; border-left: 5px solid #4CAF50; padding: 15px; margin: 20px 0px; border-radius: 5px;">
<p>A single frame is a snapshot in time. To understand a player's trajectory and recent behavior, we need to look at the immediate past. We compute lag features (the value of a variable at a previous timestep) and rolling window statistics (mean/std over the last N frames).</p>
<p>This gives the model a sense of the player's recent velocity, acceleration changes, and stability of movement, which is crucial for predicting their next move.</p>
</div>

In [ ]:
def add_sequential_features(df: pd.DataFrame) -> pd.DataFrame:
    """Computes lag and rolling window features for time-series data."""
    df = df.sort_values(["game_id", "play_id", "nfl_id", "frame_id"]).copy()
    group_cols = ["game_id", "play_id", "nfl_id"]
    
    # Columns for which to create sequential features
    seq_cols = ['x', 'y', 's', 'a', 'velocity_x', 'velocity_y']
    
    print("Creating lag features...")
    for lag in tqdm([1, 2, 3]):
        for col in seq_cols:
            if col in df.columns:
                df[f'{col}_lag{lag}'] = df.groupby(group_cols)[col].shift(lag)

    print("Creating rolling window features...")
    for window in tqdm([3, 5]):
        for col in seq_cols:
            if col in df.columns:
                # Use .transform for efficient grouped operations
                rolling_mean = df.groupby(group_cols)[col].transform(lambda s: s.rolling(window, min_periods=1).mean())
                rolling_std = df.groupby(group_cols)[col].transform(lambda s: s.rolling(window, min_periods=1).std())
                df[f'{col}_rolling_mean_{window}'] = rolling_mean
                df[f'{col}_rolling_std_{window}'] = rolling_std
    
    # Calculate change (delta) from the previous frame
    for col in ['velocity_x', 'velocity_y']:
        if col in df.columns:
            df[f'{col}_delta'] = df.groupby(group_cols)[col].diff().fillna(0.0)
            
    return df

print("Applying sequential feature engineering...")
train_input_df = add_sequential_features(train_input_df)
print("Done. Lag and rolling features added.")

3.3. GNN-lite: Modeling Player Interactions
<div style="background-color: #1F2937; color: #D1D5DB; border-left: 10px solid #F59E0B; padding: 25px; margin: 20px 0px; border-radius: 8px;">
<h3 style="color: #FBBF24; border-bottom: 2px solid #F59E0B; padding-bottom: 10px;">Why Model Interactions?</h3>
<p>A player's movement isn't made in a vacuum. A receiver adjusts their route based on the defender, and a defender reacts to the receiver. A full GNN is complex, but we can capture the essence of this with a "GNN-lite" approach. For each player (the "ego" player), we will create features that summarize the state of their local neighborhood.</p>
<h4>Our GNN-lite Process:</h4>
<ol>
<li><b>Identify Neighbors:</b> For each player at their last observed frame, find all other players on the field within a certain radius.</li>
<li><b>Calculate Relative State:</b> For each neighbor, compute their relative position (dx, dy) and velocity (dvx, dvy).</li>
<li><b>Apply Attention:</b> We can't treat all neighbors equally. A player 2 yards away is more influential than one 20 yards away. We use a softmax function based on distance (<code>exp(-dist / tau)</code>) to create attention weights. Closer players get higher weights.</li>
<li><b>Aggregate:</b> We compute a weighted average of the relative states, separately for teammates (allies) and opponents. This creates features like gnn_ally_dx_mean (the attention-weighted average x-position of teammates) or gnn_opp_dmin (distance to the nearest opponent).</li>
</ol>
<p>This gives the model a compact, powerful summary of the local competitive landscape for each player.</p>
</div>

In [ ]:
def compute_neighbor_embeddings(input_df: pd.DataFrame, cfg: CFG) -> pd.DataFrame:
    """
    Creates GNN-lite features by summarizing the state of nearby players
    at the last observed frame for each player.
    """
    print("Computing GNN-lite neighbor embeddings...")
    
    # We only need specific columns for this calculation to save memory
    cols_needed = [
        "game_id", "play_id", "nfl_id", "frame_id", "x", "y",
        "velocity_x", "velocity_y", "player_side"
    ]
    src_df = input_df[cols_needed].copy()

    # Get the state of each player at their last observed frame
    last_frame_df = (
        src_df.sort_values(["game_id", "play_id", "nfl_id", "frame_id"])
              .groupby(["game_id", "play_id", "nfl_id"], as_index=False)
              .tail(1)
              .rename(columns={"frame_id": "last_frame_id"})
              .reset_index(drop=True)
    )

    # Merge last_frame_df with all players in the same play at that specific frame
    # This creates pairs of (ego_player, neighbor_player)
    merged_df = last_frame_df.merge(
        src_df.rename(columns={
            "frame_id": "nb_frame_id", "nfl_id": "nfl_id_nb", "x": "x_nb", "y": "y_nb",
            "velocity_x": "vx_nb", "velocity_y": "vy_nb", "player_side": "player_side_nb"
        }),
        left_on=["game_id", "play_id", "last_frame_id"],
        right_on=["game_id", "play_id", "nb_frame_id"],
        how="left",
    )
    
    # Remove self-comparisons
    merged_df = merged_df[merged_df["nfl_id_nb"] != merged_df["nfl_id"]]

    # Calculate relative vectors and distance
    merged_df["dx"] = merged_df["x_nb"] - merged_df["x"]
    merged_df["dy"] = merged_df["y_nb"] - merged_df["y"]
    merged_df["dvx"] = merged_df["vx_nb"] - merged_df["velocity_x"]
    merged_df["dvy"] = merged_df["vy_nb"] - merged_df["velocity_y"]
    merged_df["dist"] = np.sqrt(merged_df["dx"]**2 + merged_df["dy"]**2)

    # Filter out distant neighbors
    merged_df = merged_df[merged_df["dist"] <= cfg.RADIUS_LIMIT].copy()

    # Identify allies vs opponents
    merged_df["is_ally"] = (merged_df["player_side_nb"] == merged_df["player_side"]).astype(float)

    # Rank neighbors by distance to find the closest ones
    keys = ["game_id", "play_id", "nfl_id"]
    merged_df["rank"] = merged_df.groupby(keys)["dist"].rank(method="first")
    
    # Keep only the top K neighbors
    merged_df = merged_df[merged_df["rank"] <= cfg.K_NEIGHBORS].copy()

    # --- Attention Weighting (Softmax) ---
    merged_df["attention"] = np.exp(-merged_df["dist"] / cfg.TAU)
    attention_sum = merged_df.groupby(keys)["attention"].transform("sum")
    merged_df["norm_attention"] = merged_df["attention"] / (attention_sum + 1e-9)
    
    merged_df["norm_attention_ally"] = merged_df["norm_attention"] * merged_df["is_ally"]
    merged_df["norm_attention_opp"] = merged_df["norm_attention"] * (1.0 - merged_df["is_ally"])

    # Pre-multiply features by attention weights for weighted aggregation
    for col in ["dx", "dy", "dvx", "dvy"]:
        merged_df[f"{col}_w_ally"] = merged_df[col] * merged_df["norm_attention_ally"]
        merged_df[f"{col}_w_opp"] = merged_df[col] * merged_df["norm_attention_opp"]
    
    # Separate distances for allies and opponents for min/mean stats
    merged_df["dist_ally"] = np.where(merged_df["is_ally"] > 0.5, merged_df["dist"], np.nan)
    merged_df["dist_opp"] = np.where(merged_df["is_ally"] < 0.5, merged_df["dist"], np.nan)

    # --- Aggregation ---
    agg_dict = {
        # Weighted means of relative vectors
        "gnn_ally_dx_mean": ("dx_w_ally", "sum"),
        "gnn_ally_dy_mean": ("dy_w_ally", "sum"),
        "gnn_ally_dvx_mean": ("dvx_w_ally", "sum"),
        "gnn_ally_dvy_mean": ("dvy_w_ally", "sum"),
        "gnn_opp_dx_mean": ("dx_w_opp", "sum"),
        "gnn_opp_dy_mean": ("dy_w_opp", "sum"),
        "gnn_opp_dvx_mean": ("dvx_w_opp", "sum"),
        "gnn_opp_dvy_mean": ("dvy_w_opp", "sum"),
        # Counts and distance stats
        "gnn_ally_count": ("is_ally", "sum"),
        "gnn_ally_dist_min": ("dist_ally", "min"),
        "gnn_ally_dist_mean": ("dist_ally", "mean"),
        "gnn_opp_dist_min": ("dist_opp", "min"),
        "gnn_opp_dist_mean": ("dist_opp", "mean"),
    }
    
    gnn_features = merged_df.groupby(keys).agg(**agg_dict).reset_index()
    gnn_features["gnn_opp_count"] = cfg.K_NEIGHBORS - gnn_features["gnn_ally_count"]

    # --- Add distance to N nearest players (regardless of side) ---
    nearest_dist = merged_df.loc[merged_df['rank'] <= 3].pivot_table(
        index=keys, columns='rank', values='dist'
    ).reset_index()
    nearest_dist.columns = [f"gnn_dist_rank{int(c)}" if isinstance(c, float) else c for c in nearest_dist.columns]
    
    gnn_features = gnn_features.merge(nearest_dist, on=keys, how="left")
    
    # Fill NaNs that occur when a player has no neighbors of a certain type
    gnn_features = gnn_features.fillna(0)
    
    print("GNN-lite embeddings computed.")
    return gnn_features

gnn_train_features = compute_neighbor_embeddings(train_input_df, CFG)
display(gnn_train_features.head())

4. Preparing the Training Dataset
<div style="background-color: #f0f8ff; color: #333; border-left: 5px solid #4CAF50; padding: 15px; margin: 20px 0px; border-radius: 5px;">
<p>Now we assemble our final training dataset. For each row in the output file (which represents a future frame we need to predict), we need to attach the features from the <b>last observed frame</b> of that player. This creates our (features, target) structure.</p>
<p>After merging, we'll also compute our physics baseline and the residual targets that our model will learn to predict.</p>
</div>

4.1. Physics Baseline & Residual Targets
<div style="background-color: #1F2937; color: #D1D5DB; border-left: 10px solid #F59E0B; padding: 25px; margin: 20px 0px; border-radius: 8px;">
<h3 style="color: #FBBF24; border-bottom: 2px solid #F59E0B; padding-bottom: 10px;">The Power of Residual Modeling</h3>
<p>Directly predicting coordinates like (110.5, 25.3) is hard. The model has to learn the entire field's coordinate system. A much more effective approach is to give the model a good "first guess" and ask it to predict the correction.</p>
<ul>
<li><b>Baseline Prediction:</b> Our "first guess" is a constant velocity model: <code>future_pos = last_pos + last_velocity * delta_t</code>. This is simple but captures the majority of the movement.</li>
<li><b>Residual (Target):</b> The value our ML model will actually predict is: <code>residual = true_future_pos - baseline_prediction</code>.</li>
<li><b>Final Prediction:</b> During inference, our final answer is: <code>Final Prediction = baseline_prediction + predicted_residual</code>.</li>
</ul>
<p>This transforms the problem from "predict where the player will be" to "predict how much the player will deviate from a straight line path," which is an easier and more stable learning task.</p>
</div>

In [ ]:
def create_training_dataframe(input_df, output_df, gnn_df):
    """
    Assembles the final training DataFrame by merging last observed stats
    with future target frames.
    """
    print("Assembling final training dataframe...")
    
    # Aggregate input_df to get the last observed state for each player in each play
    last_observed_state = (
        input_df.sort_values("frame_id")
                .groupby(["game_id", "play_id", "nfl_id"], as_index=False)
                .tail(1)
                .rename(columns={"frame_id": "last_frame_id"})
    )
    
    # Merge the last observed state with the future frames we need to predict
    train_df = output_df.rename(columns={"x": "target_x", "y": "target_y"}).merge(
        last_observed_state,
        on=["game_id", "play_id", "nfl_id"],
        how="left"
    )
    
    # Merge the GNN features
    train_df = train_df.merge(
        gnn_df,
        on=["game_id", "play_id", "nfl_id"],
        how="left"
    )
    
    # --- Calculate time delta and physics baseline ---
    train_df["delta_frames"] = train_df["frame_id"] - train_df["last_frame_id"]
    train_df["delta_t"] = train_df["delta_frames"] / 10.0  # Data is at 10Hz

    # Baseline prediction: new_pos = old_pos + velocity * time
    base_x = train_df["x"] + train_df["velocity_x"] * train_df["delta_t"]
    base_y = train_df["y"] + train_df["velocity_y"] * train_df["delta_t"]
    
    # Clip to be within field boundaries
    train_df["baseline_x"] = np.clip(base_x, 0.0, 120.0)
    train_df["baseline_y"] = np.clip(base_y, 0.0, 53.3)

    # --- Calculate residual targets ---
    train_df["residual_x"] = train_df["target_x"] - train_df["baseline_x"]
    train_df["residual_y"] = train_df["target_y"] - train_df["baseline_y"]
    
    # Evaluate the baseline's performance
    baseline_rmse = np.sqrt(0.5 * (
        mean_squared_error(train_df["target_x"], train_df["baseline_x"]) +
        mean_squared_error(train_df["target_y"], train_df["baseline_y"])
    ))
    print(f"Physics Baseline RMSE: {baseline_rmse:.5f}")
    
    return train_df

train_df = create_training_dataframe(train_input_df, train_output_df, gnn_train_features)

# Free up memory
del train_input_df, train_output_df, gnn_train_features
gc.collect()

display(train_df.head())

5. Model Training with GroupKFold CV
<div style="background-color: #f0f8ff; color: #333; border-left: 5px solid #4CAF50; padding: 15px; margin: 20px 0px; border-radius: 5px;">
<p>We are now ready to train our models. We will train two separate CatBoost models: one to predict residual_x and one for residual_y.</p>
<h4>Why GroupKFold?</h4>
<p>Standard KFold cross-validation randomly splits the data. In our case, this is dangerous. A single player's trajectory (game_id, play_id, nfl_id) is split across multiple future frames. If we randomly split, frame 30 might be in the training set while frame 40 (from the same play and player) is in the validation set. The model could "cheat" by learning specifics of that single trajectory, leading to an inflated validation score that doesn't reflect true performance on unseen plays.

GroupKFold solves this by ensuring that all rows belonging to the same group (in our case, a unique game_id-play_id-nfl_id combination) are kept together in the same fold. This simulates a more realistic scenario where the model must predict for entirely new plays.</p>
</div>

In [ ]:
def get_feature_list(df):
    """Defines the list of features to be used for training."""
    # Start with a base list of engineered features
    base_features = [
        "x", "y", "s", "a", "o", "dir", "height_inches", "bmi", "kinetic_energy", "momentum",
        "velocity_x", "velocity_y", "acceleration_x", "acceleration_y",
        "heading_x", "heading_y", "dist_to_ball", "angle_to_ball_rad",
        "velocity_towards_ball", "heading_alignment_ball", "is_offense", "is_targeted_receiver",
        "delta_t", "delta_frames"
    ]
    
    # Add lag, rolling, and delta features dynamically
    lag_cols = [c for c in df.columns if '_lag' in c]
    roll_cols = [c for c in df.columns if '_rolling_' in c]
    delta_cols = [c for c in df.columns if '_delta' in c]
    
    # Add GNN features dynamically
    gnn_cols = [c for c in df.columns if c.startswith('gnn_')]
    
    # Combine all feature lists
    all_features = base_features + lag_cols + roll_cols + delta_cols + gnn_cols
    
    # Ensure no duplicates and all columns exist in the DataFrame
    final_features = sorted(list(set([c for c in all_features if c in df.columns])))
    
    print(f"Total features to be used: {len(final_features)}")
    return final_features

# --- Prepare data for training ---
features = get_feature_list(train_df)
train_df = train_df.dropna(subset=features + ["residual_x", "residual_y"]).reset_index(drop=True)

# Replace any remaining infs/nans in features with 0
for col in features:
    train_df[col] = train_df[col].replace([np.inf, -np.inf], np.nan).fillna(0)

X = train_df[features].values.astype(np.float32)
y_x = train_df["residual_x"].values.astype(np.float32)
y_y = train_df["residual_y"].values.astype(np.float32)

# For validation scoring, we need the original targets and baselines
y_target_x = train_df["target_x"].values
y_target_y = train_df["target_y"].values
baseline_x = train_df["baseline_x"].values
baseline_y = train_df["baseline_y"].values

# Create groups for GroupKFold
groups = pd.factorize(
    train_df["game_id"].astype(str) + "_" +
    train_df["play_id"].astype(str) + "_" +
    train_df["nfl_id"].astype(str)
)[0]

# --- CV and Training Loop ---
models_x, models_y, oof_rmse_scores = [], [], []

if CFG.USE_GROUP_KFOLD:
    print(f"\nStarting training with GroupKFold (n_splits={CFG.N_FOLDS})...")
    cv = GroupKFold(n_splits=CFG.N_FOLDS)
    folds = cv.split(X, groups=groups)
else:
    print(f"\nStarting training with KFold (n_splits={CFG.N_FOLDS})...")
    cv = KFold(n_splits=CFG.N_FOLDS, shuffle=True, random_state=CFG.SEED)
    folds = cv.split(X)

for i, (train_idx, val_idx) in enumerate(folds):
    fold_num = i + 1
    print(f"\n----- Fold {fold_num}/{CFG.N_FOLDS} -----")
    
    # Split data
    X_train, X_val = X[train_idx], X[val_idx]
    yx_train, yx_val = y_x[train_idx], y_x[val_idx]
    yy_train, yy_val = y_y[train_idx], y_y[val_idx]

    # CatBoost parameters
    params = {
        'iterations': CFG.ITERATIONS,
        'learning_rate': CFG.LEARNING_RATE,
        'depth': CFG.DEPTH,
        'l2_leaf_reg': CFG.L2_REG,
        'loss_function': 'RMSE',
        'random_seed': CFG.SEED + fold_num, # Vary seed per fold
        'verbose': 500,
        'early_stopping_rounds': CFG.EARLY_STOPPING_ROUNDS,
        'task_type': 'GPU' if CFG.USE_GPU else 'CPU',
        'devices': '0' if CFG.USE_GPU else None,
    }

    # --- Train X-Model ---
    print("Training model for residual_x...")
    model_x = CatBoostRegressor(**params)
    model_x.fit(X_train, yx_train, eval_set=(X_val, yx_val))
    
    # --- Train Y-Model ---
    print("\nTraining model for residual_y...")
    model_y = CatBoostRegressor(**params)
    model_y.fit(X_train, yy_train, eval_set=(X_val, yy_val))

    # --- Validation ---
    pred_rx_val = model_x.predict(X_val)
    pred_ry_val = model_y.predict(X_val)
    
    # Add residuals back to baseline to get absolute coordinates
    pred_x_abs = baseline_x[val_idx] + pred_rx_val
    pred_y_abs = baseline_y[val_idx] + pred_ry_val
    
    # Calculate combined RMSE on absolute coordinates
    rmse_x = mean_squared_error(y_target_x[val_idx], pred_x_abs, squared=False)
    rmse_y = mean_squared_error(y_target_y[val_idx], pred_y_abs, squared=False)
    fold_rmse = np.sqrt(0.5 * (rmse_x**2 + rmse_y**2))
    
    print(f"\nFold {fold_num} Validation RMSE (absolute): {fold_rmse:.5f}")
    
    models_x.append(model_x)
    models_y.append(model_y)
    oof_rmse_scores.append(fold_rmse)

print("\n--- Training Summary ---")
print(f"Mean CV RMSE: {np.mean(oof_rmse_scores):.5f}")
print(f"Std CV RMSE:  {np.std(oof_rmse_scores):.5f}")

# Save models and features list for inference
artifacts = {
    "models_x": models_x,
    "models_y": models_y,
    "features": features,
    "cv_scores": oof_rmse_scores
}
with open(CFG.SAVE_DIR / "model_artifacts.pkl", "wb") as f:
    pickle.dump(artifacts, f)

print(f"\nModels and artifacts saved to {CFG.SAVE_DIR / 'model_artifacts.pkl'}")

6. Inference and Submission
<div style="background-color: #f0f8ff; color: #333; border-left: 5px solid #4CAF50; padding: 15px; margin: 20px 0px; border-radius: 5px;">
<p>The final step is to generate predictions on the test set. The process mirrors our training data preparation exactly:</p>
<ol>
<li>Load the test input data.</li>
<li>Apply the <b>exact same</b> feature engineering pipeline (physics, sequential, GNN-lite).</li>
<li>Create the test rows by merging the last observed state with the submission template.</li>
<li>Calculate the physics baseline for the test set.</li>
<li>Predict the residual_x and residual_y for each test row using our ensemble of trained models (averaging predictions across all folds).</li>
<li>Add the predicted residuals to the baseline to get the final coordinates.</li>
<li>Format the results into submission.csv.</li>
</ol>
<p>It is absolutely critical that the feature engineering process is identical between training and testing to avoid any data skew.</p>
</div>

In [ ]:
print("Starting inference process...")

# --- Load Test Data ---
test_input_df = pd.read_csv(CFG.BASE_DIR / "test_input.csv")
submission_template_df = pd.read_csv(CFG.BASE_DIR / "test.csv")

# --- Apply Feature Engineering Pipeline ---
print("Applying feature engineering to test data...")
test_input_df = add_physics_features(test_input_df)
test_input_df = add_sequential_features(test_input_df)
gnn_test_features = compute_neighbor_embeddings(test_input_df, CFG)

# --- Assemble Test DataFrame ---
print("Assembling test dataframe...")
last_observed_test = (
    test_input_df.sort_values("frame_id")
             .groupby(["game_id", "play_id", "nfl_id"], as_index=False)
             .tail(1)
             .rename(columns={"frame_id": "last_frame_id"})
)

test_df = submission_template_df.merge(last_observed_test, on=["game_id", "play_id", "nfl_id"], how="left")
test_df = test_df.merge(gnn_test_features, on=["game_id", "play_id", "nfl_id"], how="left")

# --- Calculate Baseline and Features ---
test_df["delta_frames"] = test_df["frame_id"] - test_df["last_frame_id"]
test_df["delta_t"] = test_df["delta_frames"] / 10.0

# Ensure all feature columns from training exist in the test set
for col in features:
    if col not in test_df.columns:
        test_df[col] = 0.0

# Clean and order columns
test_df[features] = test_df[features].replace([np.inf, -np.inf], np.nan).fillna(0.0)
X_test = test_df[features].values.astype(np.float32)

# Calculate physics baseline for test set
test_baseline_x = (test_df["x"] + test_df["velocity_x"] * test_df["delta_t"]).values
test_baseline_y = (test_df["y"] + test_df["velocity_y"] * test_df["delta_t"]).values

# --- Predict Residuals (Ensemble) ---
print("Predicting residuals with model ensemble...")
preds_rx = np.mean([model.predict(X_test) for model in models_x], axis=0)
preds_ry = np.mean([model.predict(X_test) for model in models_y], axis=0)

# --- Final Prediction ---
print("Calculating final coordinates...")
final_x = np.clip(test_baseline_x + preds_rx, 0.0, 120.0)
final_y = np.clip(test_baseline_y + preds_ry, 0.0, 53.3)

In [ ]:
print(submission_template_df.columns.tolist())

submission_df = pd.DataFrame({
    "id": (
        submission_template_df["game_id"].astype(str) + "_" +
        submission_template_df["play_id"].astype(str) + "_" +
        submission_template_df["nfl_id"].astype(str) + "_" +
        submission_template_df["frame_id"].astype(str)
    ),
    "x": final_x,
    "y": final_y
})

submission_df.to_csv("submission.csv", index=False)
submission_df.head()

<div style="background-color: #111827; color: #F3F4F6; border-left: 10px solid #3B82F6; padding: 25px; margin: 20px 0px; border-radius: 8px; box-shadow: 0 4px 8px 0 rgba(0,0,0,0.2); position: relative;">
  <h2 style="color: #60A5FA; border-bottom: 2px solid #3B82F6; padding-bottom: 10px;">Summary of Our Approach</h2>
  <p>In this notebook, we developed a robust, end-to-end pipeline for the NFL player trajectory prediction task. Our strategy, centered on <b>residual prediction on top of a physics baseline</b>, allowed a powerful model like CatBoost to focus on learning the complex, non-linear interactions and deviations from simple motion.</p>
  <p>The key components were:</p>
  <ul>
    <li><b>Meticulous Feature Engineering:</b> We created a rich feature set covering player physics, target geometry, movement history, and, crucially, local player interactions via a <b>GNN-lite embedding</b>.</li>
    <li><b>Strong Validation:</b> Using <code>GroupKFold</code> prevented data leakage and gave us a more reliable estimate of our model's true performance.</li>
    <li><b>Efficient Modeling:</b> CatBoost provided excellent performance with its handling of numerical features and robust implementation, accelerated by GPU usage.</li>
  </ul>

  <h3 style="color: #60A5FA; margin-top: 20px;">Potential Improvements & Next Steps</h3>
  <p>While this is a strong baseline, there are many avenues for improvement:</p>
  <ul>
    <li><b>More Sophisticated GNNs:</b> Our GNN-lite is effective, but a full graph convolutional network could learn more complex interaction patterns by passing information between players over multiple "hops."</li>
    <li><b>Different Model Architectures:</b> Transformer-based models, which excel at sequence modeling, could be adapted to treat a player's trajectory as a sequence and potentially capture long-range dependencies better than lags/rolling windows. LSTMs or GRUs are also classic choices for time-series data.</li>
    <li><b>Advanced Target Engineering:</b> Instead of predicting the residual (dx, dy), one could try predicting changes in velocity and acceleration, then integrate them forward in time.</li>
    <li><b>Hyperparameter Tuning:</b> A systematic search for the optimal parameters for CatBoost (e.g., using Optuna or Hyperopt) could yield further performance gains.</li>
    <li><b>Play-level Features:</b> Engineering features that describe the overall state of the play (e.g., time since snap, down, distance to go, type of pass play) could provide valuable context to the model.</li>
  </ul>

  <p>Thank you for following along! I hope this detailed walkthrough provides a solid foundation for your own experiments and success in the competition.</p>

  <p style="position: absolute; bottom: 10px; right: 20px; color: #9CA3AF; font-size: 14px; font-style: italic;">Created by Ozan M.</p>
</div>